In [51]:
# Install dependencies
%pip install -q amplpy

In [52]:
# Google Colab & Kaggle integration
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["highs"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

Using default Community Edition License for Colab. Get yours at: https://ampl.com/ce
Licensed to AMPL Community Edition License for the AMPL Model Colaboratory (https://colab.ampl.com).


In [53]:
#programação de metas
%%writefile programacaometas.mod

var X1 >= 0;
var X2 >= 0;
var Z >= 0;

  #funcao objetivo
maximize fo: Z;

  #restricoes
subject to R1:
  5 * X1 + 10 * X2 >= 60;

subject to R2:
  4 * X1 + 5 * X2 >= 35;

subject to R3:
  3 * X1 + 7 * X2 >= 40;

subject to R4:
  60 * X1 + 100 * X2 <= 600;

subject to R5:
  Z >= 0;

subject to R6:
  Z =  0 * X1 + 0 * X2;

Overwriting programacaometas.mod


In [54]:
#printando Z, x1 e x2
%%ampl_eval
reset;
model programacaometas.mod;
option solver highs;
solve;
display Z;
display X1;
display X2;
reset;

HiGHS 1.5.3: HiGHS 1.5.3: infeasible problem
3 simplex iterations
0 barrier iterations
 
Z = 0

X1 = 0

X2 = 0



In [55]:
#Método dos pesos (ponderação)
%%writefile programacaometas.mod

#SiN = quantidade que falta em relação à meta i
#SiP = quantidade de excesso em relação à meta i

var X1 >= 0;
var X2 >= 0;
var S1N >= 0;
var S1P >= 0;
var S2N >= 0;
var S2P >= 0;
var S3N >= 0;
var S3P >= 0;
var Z >= 0;

  #funcao objetivo
minimize fo: Z;

  #restricoes
subject to R1:
  5 * X1 + 10 * X2 + S1N - S1P = 60;

subject to R2:
  4 * X1 + 5 * X2 + S2N - S2P = 35;

subject to R3:
  3 * X1 + 7 * X2 + S3N - S3P = 40;

subject to R4:
  60 * X1 + 100 * X2 <= 600;

  #minimizando as faltas da meta
subject to R5:
  Z = 100 * S1N + 50 * S2N + 200 * S3N;

Overwriting programacaometas.mod


In [56]:
%%ampl_eval
reset;
model programacaometas.mod;
option solver highs;
solve;
display Z;
display X1;
display X2;
display S1N;
display S1P;
display S2N;
display S2P;
display S3N;
display S3P;
reset;

HiGHS 1.5.3: HiGHS 1.5.3: optimal solution; objective 250
4 simplex iterations
0 barrier iterations
 
Z = 250

X1 = 0

X2 = 6

S1N = 0

S1P = 0

S2N = 5

S2P = 0

S3N = 0

S3P = 2



In [57]:
#Método hierarquica - passo 1
%%writefile programacaometas.mod

# p1 = s1 ; p2 = s2 ; p3 = s3
# sendo p3 > p1 > p2 (apenas neste caso)

# vamos minimizar p3

var X1 >= 0;
var X2 >= 0;
var S1N >= 0;
var S1P >= 0;
var S2N >= 0;
var S2P >= 0;
var S3N >= 0;
var S3P >= 0;
var Z >= 0;

  #funcao objetivo
minimize fo: Z;

  #restricoes
subject to R1:
  5 * X1 + 10 * X2 + S1N - S1P = 60;

subject to R2:
  4 * X1 + 5 * X2 + S2N - S2P = 35;

subject to R3:
  3 * X1 + 7 * X2 + S3N - S3P = 40;

subject to R4:
  60 * X1 + 100 * X2 <= 600;

subject to R5:
  Z = S3N;

Overwriting programacaometas.mod


In [58]:
#Método hierarquica - passo 2
%%writefile programacaometas.mod

# agora vamos minimizar p1, que está em segunda prioridade
# lembrando p3 > p1 > p2 (apenas neste caso)

var X1 >= 0;
var X2 >= 0;
var S1N >= 0;
var S1P >= 0;
var S2N >= 0;
var S2P >= 0;
var S3N >= 0;
var S3P >= 0;
var Z >= 0;

  #funcao objetivo
minimize fo: Z;

  #restricoes
subject to R1:
  5 * X1 + 10 * X2 + S1N - S1P = 60;

subject to R2:
  4 * X1 + 5 * X2 + S2N - S2P = 35;

subject to R3:
  3 * X1 + 7 * X2 + S3N - S3P = 40;

subject to R4:
  60 * X1 + 100 * X2 <= 600;

subject to R5:
  S3N = 0;

subject to R6:
  Z = S1N;

Overwriting programacaometas.mod


In [59]:
#Método hierarquica - passo 3
%%writefile programacaometas.mod

# por último vamos minimizar p2
# lembrando p3 > p1 > p2 (apenas neste caso)

var X1 >= 0;
var X2 >= 0;
var S1N >= 0;
var S1P >= 0;
var S2N >= 0;
var S2P >= 0;
var S3N >= 0;
var S3P >= 0;
var Z >= 0;

  #funcao objetivo
minimize fo: Z;

  #restricoes
subject to R1:
  5 * X1 + 10 * X2 + S1N - S1P = 60;

subject to R2:
  4 * X1 + 5 * X2 + S2N - S2P = 35;

subject to R3:
  3 * X1 + 7 * X2 + S3N - S3P = 40;

subject to R4:
  60 * X1 + 100 * X2 <= 600;

subject to R5:
  S3N = 0;

subject to R6:
  S1N = 0;

subject to R7:
  Z = S2N;

Overwriting programacaometas.mod


In [60]:
%%ampl_eval
reset;
model programacaometas.mod;
option solver highs;
solve;
display Z;
display X1;
display X2;
display S1N;
display S1P;
display S2N;
display S2P;
display S3N;
display S3P;
reset;

HiGHS 1.5.3: HiGHS 1.5.3: optimal solution; objective 5
5 simplex iterations
0 barrier iterations
 
Z = 5

X1 = 0

X2 = 6

S1N = 0

S1P = 0

S2N = 5

S2P = 0

S3N = 0

S3P = 2

